# Transfer FGSM attack generator - Surrogate Model: InceptionResnetV1 (VGGFace2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
!pip install facenet-pytorch

In [ ]:
# Imports
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1

In [ ]:
# Loading the surrogate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mtcnn = MTCNN(keep_all=True, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
# Helper functions
def clip_box(box, W, H):
    x1, y1, x2, y2 = box
    x1 = max(0, min(W-1, x1))
    x2 = max(0, min(W-1, x2))
    y1 = max(0, min(H-1, y1))
    y2 = max(0, min(H-1, y2))
    return int(x1), int(y1), int(x2), int(y2)


def preprocess_face(face_rgb):
    face = cv2.resize(face_rgb, (160,160))
    face = torch.tensor(face/255.0, dtype=torch.float32).permute(2,0,1)
    return face.unsqueeze(0).to(device)


def fgsm_face(face_rgb, epsilon):
    x = preprocess_face(face_rgb)
    x.requires_grad = True

    emb = model(x)
    loss = emb.norm()
    loss.backward()

    grad = x.grad.data
    adv = x + epsilon * grad.sign()
    adv = torch.clamp(adv, 0, 1)

    adv_np = adv[0].permute(1,2,0).detach().cpu().numpy()
    adv_np = (adv_np * 255).astype(np.uint8)

    return adv_np


def paste_face(orig_rgb, adv_face, clipped_box):
    x1, y1, x2, y2 = clipped_box
    H = y2 - y1
    W = x2 - x1

    adv_resized = cv2.resize(adv_face, (W, H))
    patched = orig_rgb.copy()
    patched[y1:y2, x1:x2] = adv_resized
    return patched


def fgsm_video(input_path, output_path, epsilon=0.01, frames_to_attack=70):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_name = os.path.basename(input_path)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (W, H))

    detected_any_face = False

    count = 0
    while count < frames_to_attack:
        ret, frame_bgr = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        boxes, _ = mtcnn.detect(rgb)

        if boxes is None:
            out.write(frame_bgr)
            count += 1
            continue

        detected_any_face = True

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cx1, cy1, cx2, cy2 = clip_box((x1, y1, x2, y2), W, H)

            if cx2 <= cx1 or cy2 <= cy1:
                continue

            face = rgb[cy1:cy2, cx1:cx2]
            if face.size == 0:
                continue

            adv_face = fgsm_face(face, epsilon)
            rgb = paste_face(rgb, adv_face, (cx1, cy1, cx2, cy2))

        adv_bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        out.write(adv_bgr)
        count += 1

    cap.release()
    out.release()

    if not detected_any_face: # Make sure that vids without detecting a face throughout is considered invalid
        print(f"No face detected in entire video: {video_name}")


In [ ]:
# src and dst folders (Change it to your path for the manipulated videos and where you want to store the resultant videos)
src_dir = "/content/drive/MyDrive/faceforensics++/manipulated_sequences/Deepfakes/c40/videos/"
output_dir = "/content/drive/MyDrive/faceforensics++/Adversarial_attacked_sequences/TransferAttacks/FGSM//VGGFace2/InceptionResnetV1/Deepfakes/"

In [ ]:
def FGSM(epsilon):
    outdir = os.path.join(output_dir, f"Epsilon{epsilon}")
    os.makedirs(outdir, exist_ok=True)

    for video_path in tqdm(glob.glob(os.path.join(src_dir, "*.mp4"))):
        fname = os.path.basename(video_path)
        save_path = os.path.join(outdir, fname)

        if os.path.exists(save_path):
            continue

        fgsm_video(video_path, save_path, epsilon)


In [ ]:
FGSM(0.1)

  5%|▌         | 54/1000 [10:35<3:09:56, 12.05s/it]